
# Análise de modificações do modif.dat


Para realizar a análise do modif.dat será utilizado o módulo plotly



In [24]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
from datetime import datetime

pio.templates.default = "ggplot2"

O modif.dat é principal arquivo de entrada do NEWAVE para alterações cadastrais
relacioanadas a usinas hidrelétricas. Também nele são encontradas algumas restrições
físicas e operativas das usinas, que frequentemente precisam ser alteradas.
Assim, este exemplo contém uma análise e edição nas restrições operativas
de uma UHE. O modif.dat é o único arquivo modelado com a abordagem de registros
que é exclusivo do modelo NEWAVE e, por isso, sua lógica é diferente dos demais.



In [25]:
from inewave.newave import Modif

arq_modif = Modif.read("./deck_newave_2024_08_inewave/modif.dat")

É possível visualizar todos os registros de um mesmo tipo existentes no arquivo
no formato de DataFrame, porém este formato é apenas para visualização.



In [26]:
arq_modif.usina(df=True)

,codigo,nome
0,1,CAMARGOS
1,2,ITUTINGA
2,4,FUNIL-GRANDE
3,6,FURNAS
4,7,M. DE MORAES
...,...,...
97,303,FICT.PEIXE A
98,302,FICT.LAJEADO
99,267,ESTREITO TOC
100,275,TUCURUI


O outro formato para visualização dos registros é a extração dos objetos em si,
que é a maneira de se realizar também a edição de seus conteúdos. Desta forma, também
é possível aplicar filtros no registro desejado.



In [27]:
arq_modif.vazmint(data_inicio=datetime(2021, 2, 1))

Para editar o arquivo modif, necessitamos frequentemente de deletar registros existentes
e de criar novos registros. Para isto, instanciamos os modelos dos registros existentes
no módulo *inewave* e adicionamos ao objeto do arquivo através dos métodos da classe.



In [28]:
from inewave.newave.modif import TURBMAXT

codigo_usina_modificada = 7

novo_registro = TURBMAXT()
novo_registro.data_inicio = datetime(2021, 2, 1)
novo_registro.turbinamento = 1000.0
arq_modif.data.add_after(
    arq_modif.usina(codigo=codigo_usina_modificada),
    novo_registro,
)

Podemos conferir se a inserção foi feita com sucesso ao listar os registros da usina
em questão:



In [29]:
registros_usinas = arq_modif.modificacoes_usina(codigo_usina_modificada)
print([r for r in registros_usinas if isinstance(r, TURBMAXT)])

Além de edições, podemos visualizar informações contidas no modif. Por exemplo, a variação
da cota de montante de JIRAU ao longo do tempo:



In [30]:
from inewave.newave.modif import CMONT

registros_cmont = [
    r for r in arq_modif.modificacoes_usina(codigo=285) if isinstance(r, CMONT)
]
datas = [r.data_inicio for r in registros_cmont]
cotas = [r.nivel for r in registros_cmont]
df_cmont = pd.DataFrame(data={"data": datas, "cota": cotas})
fig = px.line(df_cmont, x="data", y="cota")
fig